In [1]:
!pip install tensorflow-text
!pip install datasets gensim rouge pydot graphviz

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Concatenate, Dropout, Bidirectional, MultiHeadAttention
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import os
import pickle
from google.colab import drive
from datasets import load_dataset

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
     

In [6]:

# Mount Google Drive
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/model_BiLSTM_word.h5'
history_path = '/content/drive/My Drive/history_BiLSTM_word.pkl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from tensorflow.keras.utils import to_categorical

# Load dataset
dataset = load_dataset("neulab/tldr", split='train')

# Append <start> and <end> tokens to cmd_texts
cmd_texts = ["<start>" + item['cmd'] + "<end>" for item in dataset]

# Combine nl_texts and cmd_texts for tokenizer fitting
nl_texts = [item['nl'] for item in dataset]
all_texts = nl_texts + cmd_texts

# Initialize and fit tokenizer
tokenizer = Tokenizer(filters='', lower=True, split=' ')
tokenizer.fit_on_texts(all_texts)
vocab_size = len(tokenizer.word_index) + 1  # +1 for zero padding

# Prepare sequences
nl_sequences = tokenizer.texts_to_sequences(nl_texts)
cmd_sequences = tokenizer.texts_to_sequences(cmd_texts)

# Determine maximum sequence length
max_length = max(max(len(seq) for seq in nl_sequences), max(len(seq) for seq in cmd_sequences))

# Pad sequences
nl_sequences_padded = pad_sequences(nl_sequences, maxlen=max_length, padding='post')
cmd_sequences_padded = pad_sequences(cmd_sequences, maxlen=max_length, padding='post')

# Prepare decoder input data and target data
decoder_input_data = np.zeros_like(cmd_sequences_padded)
decoder_input_data[:, 1:] = cmd_sequences_padded[:,:-1]
num_classes = vocab_size  # Set num_classes to vocab_size
decoder_target_data = to_categorical(cmd_sequences_padded, num_classes=num_classes)


In [5]:
# Confirming the shapes
print(nl_sequences_padded.shape)  # Expected shape: (batch_size, sequence_length)
print(decoder_input_data.shape)   # Expected shape: (batch_size, sequence_length)
# Convert cmd_sequences_padded to categorical
cmd_sequences_padded = to_categorical(cmd_sequences_padded, num_classes=num_classes)


(6414, 34)
(6414, 34)


In [7]:
from tensorflow.keras.initializers import Orthogonal
# Check if model exists and load or create new
if os.path.exists(model_path):
    print("Loading existing model...")
    model = load_model(model_path)
    with open(history_path, 'rb') as f:
        history = pickle.load(f)
else:
    print("Creating new model...")
    embedding_dim = 128
    units = 128

    # Build the model
    encoder_inputs = Input(shape=(None,))
    encoder_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(encoder_inputs)
    encoder_lstm = Bidirectional(LSTM(units, return_sequences=True, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_emb)
    encoder_states = [Concatenate()([forward_h, backward_h]), Concatenate()([forward_c, backward_c])]

    decoder_inputs = Input(shape=(None,))
    decoder_emb = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(units * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state=encoder_states)

    # Attention
    attn_layer = MultiHeadAttention(num_heads=4, key_dim=128)
    attn_out = attn_layer(query=decoder_outputs, key=encoder_outputs, value=encoder_outputs)
    decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attn_out])

    decoder_dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
    decoder_outputs = decoder_dense(decoder_concat_input)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    # Train the model
    checkpoint = ModelCheckpoint(model_path + ".keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)

    # Now, fit the model
    history = model.fit(
        [nl_sequences_padded, decoder_input_data],  # Encoder and Decoder inputs
        decoder_target_data,                        # Target outputs
        batch_size=64,
        epochs=100,
        validation_split=0.2,
        callbacks=[checkpoint, early_stopping]
    )

    history = history.history

    # Save the model and history
    model.save(model_path + ".keras")
    with open(history_path, 'wb') as f:
        pickle.dump(history, f)

Creating new model...


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 128)      │      1,726,464 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ [(None, None, 256),    │        263,168 │ embedding[0][0]        │
│ (Bidirectional)           │ (None, 128), (None,    │                │                        │
│                           │ 128), (None, 128),     │                │                        │
│                           │ (None, 128)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 128)      │      1,726,464 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ bidirectional[0][1],   │
│                           │                        │                │ bidirectional[0][3]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 256)            │              0 │ bidirectional[0][2],   │
│ (Concatenate)             │                        │                │ bidirectional[0][4]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │        394,240 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ concatenate[0][0],     │
│                           │ 256)]                  │                │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, None, 256)      │        526,080 │ bidirectional[0][0],   │
│ (MultiHeadAttention)      │                        │                │ lstm_1[0][0],          │
│                           │                        │                │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, None, 512)      │              0 │ lstm_1[0][0],          │
│ (Concatenate)             │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, None, 13488)    │      6,919,344 │ concatenate_2[0][0]    │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 11,555,760 (44.08 MB)

 Trainable params: 11,555,760 (44.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.8390 - loss: 3.6271
Epoch 1: val_loss improved from inf to 1.45884, saving model to /content/drive/My Drive/model_BiLSTM_word.h5.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 27s 234ms/step - accuracy: 0.8396 - loss: 3.6090 - val_accuracy: 0.8906 - val_loss: 1.4588
Epoch 2/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.8966 - loss: 1.0279
Epoch 2: val_loss improved from 1.45884 to 1.08317, saving model to /content/drive/My Drive/model_BiLSTM_word.h5.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 13s 164ms/step - accuracy: 0.8966 - loss: 1.0273 - val_accuracy: 0.8909 - val_loss: 1.0832
Epoch 3/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.8950 - loss: 0.8497
Epoch 3: val_loss did not improve from 1.08317
81/81 ━━━━━━━━━━━━━━━━━━━━ 13s 157ms/step - accuracy: 0.8950 - loss: 0.8496 - val_accuracy: 0.8907 - val_loss: 1.1399
Epoch 4/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.8950 - loss: 0.8020
Epoch 4: val_loss

In [8]:
# Calculate BLEU, ROUGE, and perplexity for validation data
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import numpy as np

def evaluate_model(model, tokenizer, nl_sequences_padded, cmd_sequences_padded, max_length, vocab_size, history):
    # Prepare the input and output for validation
    val_indices = int(len(nl_sequences_padded) * 0.8)  # Assuming validation_split=0.2
    val_nl_padded = nl_sequences_padded[val_indices:]
    val_cmd_padded = cmd_sequences_padded[val_indices:]

    # Check if cmd_sequences_padded is one-hot encoded, convert if necessary
    if val_cmd_padded.ndim == 3:
        val_cmd_padded = np.argmax(val_cmd_padded, axis=-1)

    # Initialize val_decoder_input_data correctly using explicit shape and data type
    val_decoder_input_data = np.zeros((val_cmd_padded.shape[0], val_cmd_padded.shape[1]), dtype=int)
    val_decoder_input_data[:, 1:] = val_cmd_padded[:, :-1]  # Shift cmd data for decoder input

    # Ensure it's 2D and print the shape
    print(f"Validation NL input shape: {val_nl_padded.shape}")
    print(f"Validation CMD input shape: {val_decoder_input_data.shape}")
    if val_decoder_input_data.ndim != 2:
        raise ValueError("Decoder input data should be 2D. Check data preparation steps.")

    # Predict the command sequences
    predictions = model.predict([val_nl_padded, val_decoder_input_data])

    # Convert predictions to text
    predicted_texts = []
    for prediction in predictions:
        sequence = np.argmax(prediction, axis=-1)
        text = tokenizer.sequences_to_texts([sequence])
        predicted_texts.append(text[0])

    # Convert actual command sequences to text
    actual_texts = tokenizer.sequences_to_texts(val_cmd_padded)

    # Compute BLEU and ROUGE scores
    bleu_scores = [sentence_bleu([act.split()], pred.split()) for act, pred in zip(actual_texts, predicted_texts)]
    rouge = Rouge()
    rouge_scores = [rouge.get_scores(pred, act)[0] for pred, act in zip(predicted_texts, actual_texts)]

    # Calculate mean BLEU and ROUGE scores
    mean_bleu = np.mean(bleu_scores)
    rouge_f = np.mean([score['rouge-l']['f'] for score in rouge_scores])

    # Calculate perplexity on validation loss
    val_loss = history['val_loss'][-1]
    perplexity = np.exp(val_loss)

    print(f"Average BLEU Score on Validation: {mean_bleu}")
    print(f"Average ROUGE-L F-Score on Validation: {rouge_f}")
    print(f"Perplexity on Validation: {perplexity}")

# Ensure history is correctly passed to the function after the model has been trained
evaluate_model(model, tokenizer, nl_sequences_padded, cmd_sequences_padded, max_length, vocab_size, history)


Validation NL input shape: (1283, 34)
Validation CMD input shape: (1283, 34)
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step
Average BLEU Score on Validation: 0.00019935040507600187
Average ROUGE-L F-Score on Validation: 0.03405840599370301
Perplexity on Validation: 3.475041503589121


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [9]:
def translate(model, tokenizer, text, max_length):
    # Tokenize the input text
    sequence = tokenizer.texts_to_sequences([text])
    sequence_padded = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Prepare the decoder input data
    decoder_input_data = np.zeros_like(sequence_padded)
    decoder_input_data[:, 1:] = sequence_padded[:, :-1]

    # Predict
    prediction = model.predict([sequence_padded, decoder_input_data])

    # Convert prediction to text
    predicted_sequence = np.argmax(prediction[0], axis=-1)
    predicted_text = tokenizer.sequences_to_texts([predicted_sequence])

    # Remove the <start> and <end> tokens
    predicted_text_clean = [token for token in predicted_text[0].split() if token not in ['<start>', '<end>']]

    # Join the tokens back together
    predicted_command = ' '.join(predicted_text_clean)

    return predicted_command

In [10]:
# Get the first 10 rows of the training dataset
first_10_nl_texts = nl_texts[:10]
first_10_cmd_texts = cmd_texts[:10]

# Translate each text and print the input, prediction, and actual command
for i in range(10):
    input_text = first_10_nl_texts[i]
    actual_command = first_10_cmd_texts[i]
    predicted_command = translate(model, tokenizer, input_text, max_length)

    print(f"Input Text: {input_text}")
    print(f"Predicted Command: {predicted_command}")
    print(f"Actual Command: {actual_command}")
    print("\n---\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
Input Text: get the label of a fat32 partition
Predicted Command: <start>sudo install {{path/to/file}}<end> {{path/to/file}}<end>
Actual Command: <start>fatlabel {{/dev/sda1}}<end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Input Text: set the label of a fat32 partition
Predicted Command: <start>sudo pacman -s {{path/to/file}}<end> > > >
Actual Command: <start>fatlabel {{/dev/sdc3}} "{{new_label}}"<end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Input Text: search for a package in your current sources
Predicted Command: <start>sudo -c
Actual Command: <start>apt-cache search {{query}}<end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Input Text: show information about a package
Predicted Command: <start>sudo list<end>
Actual Command: <start>apt-cache show {{package}}<end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input Text: show whether a package is installed and up to date
Predicted Command: <start>git diff
Actual Command: <start>apt-cache pol